In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import warnings
import re

In [2]:
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LassoCV,LassoLarsIC,Lasso,LinearRegression

In [3]:
def lasso_moran_select(data,s):
    
    X_coords = data.iloc[:,:-3]
    
    y = data['y']
    
    X1 = data.X1.values
    X2 = data.X2.values
    
    moran = data.iloc[:,2:-3].values
    
    moran_X1_names = ["moran_X1_" + str(i) for i in range(moran.shape[1])]
    X_coords.loc[:,moran_X1_names] = moran*X1.reshape(-1,1)
    
    moran_X2_names = ["moran_X2_" + str(i) for i in range(moran.shape[1])]
    X_coords.loc[:,moran_X2_names] = moran*X2.reshape(-1,1)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    if s == 'mse':
        model = LassoCV(cv=kf,  alphas=np.logspace(-4, -2, 20))
        model.fit(X_coords, y)
        r2_scores = cross_val_score(model, X_coords, y, cv=kf, scoring='r2')
        
        print("MSE LASSO CV R2 score: ", np.mean(r2_scores))
        
    if s == 'bic':
        model = LassoLarsIC(criterion='bic')
        model.fit(X_coords, y)
        
        r2_scores = cross_val_score(model, X_coords, y, cv=kf, scoring='r2')
        
        print("BIC LASSO CV R2 score: ", np.mean(r2_scores))
        

    selected_index = np.where(model.coef_ != 0)[0]
    selected_feature = list(X_coords.columns[selected_index])

    moran_X1s = [s for s in selected_feature if s.startswith('moran_X1_')]
    moran_X2s = [s for s in selected_feature if s.startswith('moran_X2_')]
    moran_s = [s for s in selected_feature if re.match(r"^moran_\d+$", s)]

    numbers_s = set(int(x.split('_')[1]) for x in moran_s)
    numbers_x1 = set(int(x.split('_')[2]) for x in moran_X1s)
    numbers_x2 = set(int(x.split('_')[2]) for x in moran_X2s)

    # Find intersection of numbers in both lists
    common_numbers = numbers_s.union(numbers_x1.union(numbers_x2))

    moran_selected = ['moran_' + str(num) for num in sorted(common_numbers)]
    
    return moran_selected

In [4]:
from tabnet_model import TabNet_fit

In [5]:
weights = ['queen','exp'] #2
models = ['tabnet'] #1
selection = ['xy','all','mse','bic'] #4

In [6]:
warnings.filterwarnings('ignore')

for w in weights:
    data = pd.read_csv('data/grid_moran_' + w + '.csv',index_col=0)
    
    for s in selection:
        
        if s == 'all':
            selected = list(data.columns[2:-3])
        if s == 'xy':
            selected = list(data.columns[-3: -1])
        if s =='mse':
            selected = lasso_moran_select(data,s)
        if s == 'bic':
            selected = lasso_moran_select(data,s)
        if s == 'aic':
            selected = lasso_moran_select(data,s)
            
        
        X_coords = data[['X1','X2']  + selected]
        
        moran = X_coords[selected]
        moran = (np.array(moran) - np.array(moran).mean(axis=0))/np.array(moran).std(axis=0)
        X_coords[selected] = moran
        
        y = data['y']

        for m in models:
            
            model,best_loss = TabNet_fit(X_coords.values, y.values.reshape(-1,1), timeout=60*30)
            
            #model.fit(X_coords, y, estimator_list = [m], **settings)
            
            name = "models/grid_" + m + "_moran_" + s + "_lasso_" + w + ".model"
            
            pickle.dump(model, open(name, 'wb'))
            
            print(name + ' CV-R2 = {0:.3g}'.format(best_loss))

[I 2024-05-09 17:29:02,180] A new study created in memory with name: no-name-7a7ad317-a2ea-4621-8c3b-67d425945f61



Early stopping occurred at epoch 362 with best_epoch = 290 and best_val_0_rmse = 0.79886
Stop training because you reached max_epochs = 369 with best_epoch = 328 and best_val_0_rmse = 0.71172
Stop training because you reached max_epochs = 369 with best_epoch = 363 and best_val_0_rmse = 0.77065
Stop training because you reached max_epochs = 369 with best_epoch = 357 and best_val_0_rmse = 0.82278


[I 2024-05-09 17:31:10,141] Trial 0 finished with value: -0.9465930317783684 and parameters: {'mask_type': 'sparsemax', 'n_da': 64, 'n_steps': 3, 'gamma': 1.1, 'n_shared': 1, 'lambda_sparse': 1.951722464144948e-08, 'patienceScheduler': 14, 'patience': 72, 'epochs': 369}. Best is trial 0 with value: -0.9465930317783684.


Stop training because you reached max_epochs = 369 with best_epoch = 366 and best_val_0_rmse = 0.79889
Stop training because you reached max_epochs = 181 with best_epoch = 176 and best_val_0_rmse = 1.07735
Stop training because you reached max_epochs = 181 with best_epoch = 167 and best_val_0_rmse = 0.86404
Stop training because you reached max_epochs = 181 with best_epoch = 170 and best_val_0_rmse = 1.16619
Stop training because you reached max_epochs = 181 with best_epoch = 180 and best_val_0_rmse = 0.95872


[I 2024-05-09 17:31:58,202] Trial 1 finished with value: -0.9114547178890305 and parameters: {'mask_type': 'sparsemax', 'n_da': 64, 'n_steps': 2, 'gamma': 1.1, 'n_shared': 1, 'lambda_sparse': 3.3205591037519537e-07, 'patienceScheduler': 9, 'patience': 60, 'epochs': 181}. Best is trial 0 with value: -0.9465930317783684.


...


Early stopping occurred at epoch 370 with best_epoch = 279 and best_val_0_rmse = 0.8449

Early stopping occurred at epoch 236 with best_epoch = 150 and best_val_0_rmse = 0.95049

Early stopping occurred at epoch 215 with best_epoch = 129 and best_val_0_rmse = 0.89841

Early stopping occurred at epoch 245 with best_epoch = 159 and best_val_0_rmse = 1.02271

Early stopping occurred at epoch 245 with best_epoch = 159 and best_val_0_rmse = 1.00869


[I 2024-05-09 21:51:20,396] Trial 13 finished with value: -0.9226339463101993 and parameters: {'mask_type': 'entmax', 'n_da': 60, 'n_steps': 1, 'gamma': 1.0, 'n_shared': 3, 'lambda_sparse': 1.984329576941398e-05, 'patienceScheduler': 12, 'patience': 86, 'epochs': 340}. Best is trial 6 with value: -0.9325928550469245.



Early stopping occurred at epoch 277 with best_epoch = 191 and best_val_0_rmse = 0.81248

Early stopping occurred at epoch 246 with best_epoch = 203 and best_val_0_rmse = 0.92644

Early stopping occurred at epoch 313 with best_epoch = 270 and best_val_0_rmse = 1.02771

Early stopping occurred at epoch 199 with best_epoch = 156 and best_val_0_rmse = 0.98665

Early stopping occurred at epoch 235 with best_epoch = 192 and best_val_0_rmse = 0.96584


[I 2024-05-09 21:52:59,233] Trial 14 finished with value: -0.9190754434024951 and parameters: {'mask_type': 'entmax', 'n_da': 60, 'n_steps': 2, 'gamma': 1.2, 'n_shared': 2, 'lambda_sparse': 0.0004102780207828637, 'patienceScheduler': 13, 'patience': 43, 'epochs': 420}. Best is trial 6 with value: -0.9325928550469245.



Early stopping occurred at epoch 229 with best_epoch = 186 and best_val_0_rmse = 0.89286
models/grid_tabnet_moran_bic_lasso_exp.model CV-R2 = 0.933


### LASSO Results

In [ ]:
for w in weights:
    data = pd.read_csv('data/grid_moran_' + w + '.csv',index_col=0)
    
    for s in ['mse','bic','aic']:
        
        if s =='mse':
            selected = lasso_moran_select(data,s)
            print(len(selected))
        if s == 'bic':
            selected = lasso_moran_select(data,s)
            print(len(selected))

MSE LASSO CV R2 score:  0.7952897584979441
172


### Linear Model

In [ ]:
data = pd.read_csv('data/grid_moran_' + w + '.csv',index_col=0)
X_coords = data.iloc[:,:2]

model = LinearRegression()

# Set up cross-validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and calculate R2 scores
r2_scores = cross_val_score(model, X_coords, y, cv=kf, scoring='r2')

In [ ]:
r2_scores.mean()